#**CycleGAN**

In [1]:
import torch.nn as nn
import torch.nn.functional as F

`in_features` (int): number of input features in the first convolution

In [2]:
class Residual_block(nn.Module):
  def __init__(self, in_features):
    super(Residual_block,self).__init__()

    resi_block = [  nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True),
                        nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features)  ]

    self.resi_block= nn.Sequential(*resi_block)


  def forward(self, x):
    return self.resi_block(x)



`in_feature` (int): number of input features in the first convolution

`out_feature` (int): number of output features

In [3]:
class Conv_block(nn.Module):
  def __init__(self,in_feature, out_feature):
    super(Conv_block,self).__init__()

    conv_block= [ nn.Conv2d(in_feature, out_feature,kernel_size=3, stride=2, padding=1),
                            nn.InstanceNorm2d(out_feature),
                            nn.ReLU(inplace=True)   ]

    self.conv_block= nn.Sequential(*conv_block)


  def forward(self, x):
    return self.conv_block(x)


`in_feature` (int): number of input features in the first convolution

`out_feature` (int): number of output features

In [4]:
class Deconv_block(nn.Module):
  def __init__(self,in_feature, out_feature):
    super(Deconv_block,self).__init__()

    deconv_block= [ nn.ConvTranspose2d(in_feature, out_feature,kernel_size=3, stride=2, padding=1),
                            nn.InstanceNorm2d(out_feature),
                            nn.ReLU(inplace=True)   ]

    self.deconv_block= nn.Sequential(*deconv_block)


  def forward(self, x):
    return self.deconv_block(x)


`in_feature` (int): number of input features in the first convolution

`num_filter` (int): number of output features at level

`out_feature` (int): number of output features

In [5]:
class Generator(nn.Module):
  def __init__(self,in_feature,num_filter, out_feature, residual_block= 3):
    super(Generator, self).__init__()

    self.pad = nn.ReflectionPad2d(3)

    self.conv1= Conv_block(in_feature, num_filter, kernel_size=7, stride=1, padding=0)
    self.conv2= Conv_block(num_filter, num_filter * 2)
    self.conv3= Conv_block(num_filter * 2, num_filter * 4)


    self.resi_block= []
    for i in range(residual_block):
      self.resi_block.append(Residual_block(num_filter * 4))

    self.resi_block= nn.Sequential(*self.resi_block)

    self.deconv1= Deconv_block(num_filter * 4, num_filter * 2)
    self.deconv2= Deconv_block(num_filter * 2, num_filter)
    self.deconv3= Deconv_block(num_filter, out_feature, kernel_size=7, stride=1, padding=0)


  def forward(self,x):
    en1= self.conv1(self.pad(x))
    en2= self.conv2(en1)
    en3= self.conv3(en2)

    res= self.resi_block(en3)

    de1= self.deconv1(res)
    de2= self.deconv2(de1)
    out= self.deconv3(de2)

    return out




`in_feature` (int): number of input features in the first convolution

`num_filter` (int): number of output features at level

`out_feature` (int): number of output features

In [6]:
class Discriminator(nn.Module):
  def __init__(self,in_feature, num_filters, out_features):
    super(Discriminator, self).__init__()

    self.conv1= Conv_block(in_feature, num_filters, kernel_size=4, stride=2, padding=1)
    self.conv2= Conv_block(num_filters, num_filters * 2,kernel_size=4, stride=2, padding=1)
    self.conv3= Conv_block(num_filters * 2, num_filters * 4,kernel_size=4, stride=2, padding=1)
    self.conv4= Conv_block(num_filters * 4, num_filters * 8,kernel_size=4, stride=2, padding=1)

    self.conv5= nn.Conv2d(num_filters * 8, out_features,kernel_size=4, stride=2, padding=1)

  def forward(self,x):

    l1= F.leaky_relu(self.conv1(x), 0.2)
    l2= F.leaky_relu(self.conv2(l1), 0.2)
    l3= F.leaky_relu(self.conv3(l2), 0.2)
    l4= F.leaky_relu(self.conv4(l3), 0.2)
    l5= self.conv5(l4)

    return l5
